In [ ]:
import itertools 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from quantopian.research.experimental import continuous_future, history


In [ ]:
def get_hist(symbol, start, end, num_of_maturities): 
    df = history(
        [continuous_future(symbol, 
                           offset=i,
                           adjustment=None) for i in range(num_of_maturities)],
        fields = 'open_price',#'price',
        frequency='daily',
        start=start,
        end=end
    )
    return df 


In [ ]:
symbols=['SY', 'BO', 'SM', 'CN'] # Soybean, oil, meal 

symbols_pairs = itertools.combinations(symbols,2)
symbols_pairs = list(symbols_pairs)
# >>> [('SY', 'BO'),('SY', 'SM'),('BO', 'SM')


In [ ]:
hist_df = [get_hist(symbol,'2013-01-01', '2018-7-12', 5) for symbol in symbols]



In [ ]:
maturities_df = [pd.concat(
    [df.iloc[:, i] for df in hist_df], axis=1)
                 for i, x in enumerate(hist_df[0].columns)]


In [ ]:
for df in maturities_df:
    df.columns = symbols 


In [ ]:

def plot_ratio(maturity):
    fig = plt.figure()
    fig.suptitle('offset:{}'.format(maturity))
    ax = [fig.add_subplot(3, 1, i) for i in range(1, 4)]
    ax  = fig.add_subplot(1,1,1)
    for i, symbol in enumerate(symbols_pairs):
        ax[i].plot(maturities_df[maturity][list(symbol)].apply(lambda x: x[0] / x[1], axis=1))
        ax[i].set_title('{}/{}'.format(*symbol))
        ax.set_title('{}/{}'.format(*symbol))

        

In [ ]:
def compare(num,):
    df = maturities_df[num]
    df = df.fillna(method='ffill')
    df = (df / df.iloc[0]).applymap(np.log)
        
    for p1, p2 in symbols_pairs:
        clm = '{}/{}'.format(p1, p2)
        df[clm] = df[p1] - df[p2]
    return df 

def compare1(num,):
    df = maturities_df[num]
    df = df.fillna(method='ffill')
    df = (df/df.shift(1)).applymap(np.log)
        
    for p1, p2 in symbols_pairs:
        clm = '{}/{}'.format(p1, p2)
        df[clm] = df[p1] - df[p2]
    return df 


group1 = compare(1)
group2 = compare(2)
group3 = compare(3)
group4 = compare(4)

g1 = compare1(1)



In [ ]:
g1[['{}/{}'.format(p1, p2) for p1, p2 in symbols_pairs]].rolling(window=30).sum().describe()


In [ ]:
g1[['{}/{}'.format(p1, p2) for p1, p2 in symbols_pairs]].rolling(window=60).sum().describe()

In [ ]:
g1[["SY/SM","SY/CN"]].rolling(window=30).sum().plot()

In [ ]:
group1[['{}/{}'.format(p1, p2) for p1, p2 in symbols_pairs]].plot()


In [ ]:
group1[['SY/BO', 'BO/SM',]].plot()
np.abs(group1['SY/BO'] - group1['BO/SM']).plot()
#group1['BO'].plot()


In [ ]:
x = g1[['SY/BO', 'BO/SM',]].rolling(window=30).sum()#.plot()
x.plot()
(x['SY/BO'] - x['BO/SM']).plot()

In [ ]:
rtn = maturities_df[1].pct_change()
flag = (x['SY/BO'] - x['BO/SM']) > 0.2 
flag2 = (x['SY/BO'] - x['BO/SM']) < -0.2
rtn['SY']-rtn['SM'] * (flag*1 + flag2*-1) 

In [ ]:
rtn = maturities_df[1].pct_change()
flag = (group1["BO/SY"] - group1["SM/SY"]) < -0.15
rtn["sm long bo short flag"] = flag
((rtn['SM'] - rtn['BO']) * rtn["sm long bo short flag"] ).cumsum().plot()

In [ ]:
((rtn['SM'] - rtn['BO']) * rtn["sm long bo short flag"] ).cumsum().plot()
(group1["BO/SY"] - group1["SM/SY"]).plot()

In [ ]:
g1.rolling(window=60).sum()[["BO/SY","SM/SY"]].plot()
    

In [ ]:
from statsmodels.tsa.stattools import coint
score, pvalue, _ = coint(x['SM'],x['SY'])
pvalue

In [ ]:
group1[["SM/SY"]] > 1

def plotSM(m_df, group):
    x = m_df.pct_change()['SM'] - m_df.pct_change()['SY']
    
    (x * (group["SM/SY"] > 1 )*1 + \
     x*-1 * (group["SM/SY"] < 1 )*1).cumsum().plot()
    
plotSM(maturities_df[1], group1)    
plotSM(maturities_df[2], group2)    
plotSM(maturities_df[3], group3)    
plotSM(maturities_df[4], group4)    

group1[[ 'SM', 'SY']].plot()#"SM/SY",

In [ ]:
def plotBO(m_df, group):
    (m_df.pct_change()['BO'] * (group["BO/SY"] > 1 )*1 + \
     m_df.pct_change()['BO']*-1 * (group["BO/SY"] < 1 )*1).cumsum().plot()

def plotSM(m_df, group):
    (m_df.pct_change()['SM'] * (group["SM/SY"] > 1 )*1 + \
     m_df.pct_change()['SM']*-1 * (group["SM/SY"] < 1 )*1).cumsum().plot()
    

plotBO(maturities_df[1], group1)  
plotBO(maturities_df[2], group2)  
plotBO(maturities_df[3], group3)  
plotBO(maturities_df[4], group4)  

# plotSM(maturities_df[1], group1)  
# plotSM(maturities_df[2], group2)  
# plotSM(maturities_df[3], group3)  
# plotSM(maturities_df[4], group4)  



In [ ]:
#group1[['BO/SY','SM/SY', ]].sum(axis=1).plot()
def plotBO1(m_df, group):
    x = m_df.pct_change()['BO']-m_df.pct_change()['SY']
    (x  * (group1[['BO/SY','SM/SY']].sum(axis=1)> 2.0 )*1 + \
     x*-1 * (group1[['BO/SY','SM/SY']].sum(axis=1)< 2.0)*1).cumsum().plot()
    
plotBO1(maturities_df[1], group1)   




In [ ]:
bo_sm_spread = group1_log["BO/SY"] - group1_log["SM/SY"]
bo_sm_spread = bo_sm_spread.apply(np.abs)
                          
group1_log['bo_sm_spread'] = bo_sm_spread
group1_log['bo_sm_spread wide'] = group1_log['bo_sm_spread'] > 0.2
group1_log[group1_log['bo_sm_spread wide']].describe()

group1_log[["BO/SY","SM/SY",]].plot() 
group1_log['bo_sm_spread wide'].plot()


In [ ]:
x = (bo_sm_spread<0.2) & (bo_sm_spread.shift(1) > 0.2)
group1_log[x]


In [ ]:
group1_log[['BO','SM']].loc["2015-04-06":].head(30).plot()

In [ ]:
bo = (group1["BO"] / group1["SY"]).apply(np.log)
sm = (group1["SM"] / group1["SY"]).apply(np.log) 
#bo.plot()
#sm.plot()
(group1["SM"] - group1["BO"]).plot()
(sm-bo).plot(secondary_y=True)


In [ ]:
df_test = hoge[['SY', 'SM']].replace([np.inf, -np.inf], np.nan).dropna()
X = df_test.loc[:, ['SY']].as_matrix()
y = df_test['SM'].as_matrix()

clf.fit(X,y)

# 回帰係数
print(clf.coef_)
 
# 切片 (誤差)
print(clf.intercept_)
 
# 決定係数
print(clf.score(X, y))

In [ ]:
import matplotlib.pyplot as plt 
plt.scatter(X,y)
plt.plot(X, clf.predict(X))

In [ ]:
df_test['X'] = X[:,0]
df_test['y'] = clf.predict(X)
df_test



In [ ]:
df_test.describe()

In [ ]:
df_test

In [ ]:
df_test['go_long'] = df_test['y'] < -1#-0.006473*1.8
df_test['go_short'] = df_test['y'] > 0.006363*1.8
df_test['SY-SM'] = df_test['SY'] - df_test['SM']

df_test["go_long"] = df_test["go_long"] * 1 
df_test["go_short"] = df_test["go_short"] * -1 
df_test["position"] = df_test["go_short"] #+df_test["go_long"] 
#df_test["position"].replace(0,np.nan).fillna(method='ffill')


In [ ]:
#(df_test["SY-SM"] * df_test['position']).cumsum().plot()
df_test["pl"] = df_test["SY-SM"] * df_test['position']


In [ ]:
plt.plot(df_test['SY'].cumsum())
plt.plot(df_test['SM'].cumsum())
plt.plot(df_test['pl'].cumsum())